# Convolutional Neural Networks for Sentence Classification by Yoon Kim in Keras

[Kim's paper](http://www.aclweb.org/anthology/D14-1181) was published in 2014 and showed that not only are CNNs great for images but also text. With an extreamly simple architecture Kim outperformed all other models in 4 out of 7 benchmarks at the time. In my first attempt to reproduce a paper on Kaggle, I'm going to try and recreate what he did with Keras.

## Architecture

![Architecture diagram](https://imgur.com/Dsu7Q61.png)

*- architecture diagram taken from Kim's paper*

The architecture Kim used goes like this:

Firstly, sentences are represented as vectors of words and those words are converted into (300D) vectors giving us a 2D representation for each sentence. Kim uses a few different approaches for creating these word vectors:

- **CNN-rand**: where embeddings are randomly assigned to each word
- **CNN-static**: word2vec is used to provide word embeddings. Unknown words are randomly initialised. These embeddings are kept fixed.
- **CNN-non-static**: as above, but the vectors are fine-tuned (ie they can be changed) during training

Kim also tried another approach which I won't attempt to do here.. because quite frankly I don't know how. I might attempt this in the future.

- **CNN-multichannel**: Here Kim uses two sets of word vectors. My understanding is he applies both to the sentence (resulting in two, sentence_length x 300D matricies) and performs convolutions on both (see below). During training he fine tunes one embedding but keeps the other one fixed. He states the hope of this was that it would act as a form of regularization but the results were mixed. I'm not sure if this approach was investigated and improved upon in other papers or not. Hopefully it was, it sounds like the kind of thing that might work.

He then performs convolutions on these 2D representations, but he does it in a way you don't see very often. Convolutions of different window sizes (3, 4 and 5) are performed on the representations directly and then max pooled. This is a little like an inception module in the Goog LeNet, but not something I've really seen anywhere else.

Finally after a layout of dropout, an output layer is directly used to do the predictions.

And that's it! With one conv layer and some max pooling we can get fantastic results!


### Why this paper is interesting

We all know LSTMs are the king of sentence classification... but are they really?

Recently I tried to classify a million sentences where I work with a (admittedly complicated) LSTM. It was estimated it would take 130 hours to run on the machine available. With something inspired by this model it takes around an hour and we achieved compariable accuracy (these are short sentences which is believe why that works).

It's also interesting to see totally different approaches to solving this problem.

**Update** while working on another problem I found myself doing something Kim does in this paper which you don't often see:

> When randomly initializing words not in word2vec, we obtained slight improvements by sampling each dimension from `U[−a, a]` where a was chosen such that the randomly initialized vectors have the same variance as the pre-trained ones. It would be interesting to see if employing more sophisticated methods to mirror the distribution of pre-trained vectors in the initialization process gives further improvements.

This is really interesting and something I would like to explore in another kernel.


### Places this differs from Kim's paper

Sadly I couldn't find the same data Kim used, but I've managed to find something very similar sounding to the SST1 data (from which I can infer the SST2 data) where it looks like we can achieve simiar results.

Kim uses a 300D w2v model trained using cbow. I couldn't find this model, but used a 200D model trained using GloVe. The major impact of this is we use an embedding dimension of 200 here.

I also swapped out the optimizer Kim used (AdaDelta) for Adam - AdaDelta took forever to run but it looks like you get the same results as Adam. If you're happy to sit there for around 100 epochs you can try it out.


# Data prep

So let's get started. Before we do anything else we need to get some data and prep it

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
# lets import some stuff
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.initializers import Constant
from keras.models import Model
from keras.layers import *
from keras.utils.np_utils import to_categorical
import re

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
df = pd.read_csv('../input/movie-review-sentiment-analysis-kernels-only/train.tsv', delimiter='\t')
df = df[['Phrase', 'Sentiment']]

pd.set_option('display.max_colwidth', -1)
df.head(3)

So, you can see that the `Phrase` data which we're going to train our model against hasn't been cleaned so we need to do it.

I am just going to do something very simple, which is to turn url's into `url`, remove anything that's not alphanumeric or a space. Then lowercase what's left.

In [ ]:
def clean_str(in_str):
    in_str = str(in_str)
    # replace urls with 'url'
    in_str = re.sub(r"(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9]\.[^\s]{2,})", "url", in_str)
    in_str = re.sub(r'([^\s\w]|_)+', '', in_str)
    return in_str.strip().lower()


df['text'] = df['Phrase'].apply(clean_str)

Our data is classified into 5 different classes, very negative, slightly negative, neutral, slightly positive and very positive.

Sadly our dataset isn't balanced, so we need to do that ourselves

In [ ]:
df.Sentiment.value_counts()

In [ ]:
df_0 = df[df['Sentiment'] == 0].sample(frac=1)
df_1 = df[df['Sentiment'] == 1].sample(frac=1)
df_2 = df[df['Sentiment'] == 2].sample(frac=1)
df_3 = df[df['Sentiment'] == 3].sample(frac=1)
df_4 = df[df['Sentiment'] == 4].sample(frac=1)

# we want a balanced set for training against - there are 7072 `0` examples
sample_size = 7072

data = pd.concat([df_0.head(sample_size), df_1.head(sample_size), df_2.head(sample_size), df_3.head(sample_size), df_4.head(sample_size)]).sample(frac=1)

Now, we need to tokenize our sentences into vectors. Before we do that we need to pick a vector size.

In [ ]:
data['l'] = data['Phrase'].apply(lambda x: len(str(x).split(' ')))
print("mean length of sentence: " + str(data.l.mean()))
print("max length of sentence: " + str(data.l.max()))
print("std dev length of sentence: " + str(data.l.std()))

In [ ]:
# these sentences aren't that long so we may as well use the whole string
sequence_length = 52

In [ ]:
max_features = 20000 # this is the number of words we care about

tokenizer = Tokenizer(num_words=max_features, split=' ', oov_token='<unw>')
tokenizer.fit_on_texts(data['Phrase'].values)

# this takes our sentences and replaces each word with an integer
X = tokenizer.texts_to_sequences(data['Phrase'].values)

# we then pad the sequences so they're all the same length (sequence_length)
X = pad_sequences(X, sequence_length)

y = pd.get_dummies(data['Sentiment']).values

# where there isn't a test set, Kim keeps back 10% of the data for testing, I'm going to do the same since we have an ok amount to play with
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

print("test set size " + str(len(X_test)))

# Model 1: Random embeddings

Lets build our model. In general I'm going to just use the same hyperparameters as Kim does (see section 3.1 of his paper) apart from the embedding dimension

Keras has an Embedding layer we can use here. If you don't specify a custom way to embed text (something we will do later with w2v) Keras will do it randomly with a normal (Gaussian) distribution for you

In [ ]:
embedding_dim = 200 # Kim uses 300 here
num_filters = 100

inputs = Input(shape=(sequence_length,), dtype='int32')

# use a random embedding for the text
embedding_layer = Embedding(input_dim=max_features, output_dim=embedding_dim, input_length=sequence_length)(inputs)

reshape = Reshape((sequence_length, embedding_dim, 1))(embedding_layer)

# Note the relu activation which Kim specifically mentions
# He also uses an l2 constraint of 3
# Also, note that the convolution window acts on the whole 200 dimensions - that's important
conv_0 = Conv2D(num_filters, kernel_size=(3, embedding_dim), activation='relu', kernel_regularizer=regularizers.l2(3))(reshape)
conv_1 = Conv2D(num_filters, kernel_size=(4, embedding_dim), activation='relu', kernel_regularizer=regularizers.l2(3))(reshape)
conv_2 = Conv2D(num_filters, kernel_size=(5, embedding_dim), activation='relu', kernel_regularizer=regularizers.l2(3))(reshape)

# perform max pooling on each of the convoluations
maxpool_0 = MaxPool2D(pool_size=(sequence_length - 3 + 1, 1), strides=(1,1), padding='valid')(conv_0)
maxpool_1 = MaxPool2D(pool_size=(sequence_length - 4 + 1, 1), strides=(1,1), padding='valid')(conv_1)
maxpool_2 = MaxPool2D(pool_size=(sequence_length - 5 + 1, 1), strides=(1,1), padding='valid')(conv_2)

# concat and flatten
concatenated_tensor = Concatenate(axis=1)([maxpool_0, maxpool_1, maxpool_2])
flatten = Flatten()(concatenated_tensor)

# do dropout and predict
dropout = Dropout(0.5)(flatten)
output = Dense(units=5, activation='softmax')(dropout)

Kim uses Adadelta as his optimizer. After some experimentation it's clear you get the same results with adam, it just happens to get there MUCH slower with adadelta (which makes sense). For everyone's sanity I'm going to use adam here - if you'd like to follow Kim's paper closer than me then all you have to do is swap `adam` for `adadelta` in the next cell and increase the number of epochs to around 100

In [ ]:
model = Model(inputs=inputs, outputs=output)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

In [ ]:
batch_size = 32 # Kim uses 50 here, I have a slightly smaller sample size than num
history = model.fit(X_train, y_train, epochs=30, batch_size=batch_size, verbose=1, validation_split=0.1, shuffle=True)

Let's plot the training

In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

as you can see this is a bit noisey, but it doesn't overfit too badly or do crazy things, so I'm happy!

So how good is this model? Lets test it with our test set

In [ ]:
y_hat = model.predict(X_test)

In [ ]:
accuracy_score(list(map(lambda x: np.argmax(x), y_test)), list(map(lambda x: np.argmax(x), y_hat)))

In [ ]:
confusion_matrix(list(map(lambda x: np.argmax(x), y_test)), list(map(lambda x: np.argmax(x), y_hat)))

This is suprisingly good - a random model would achieve a score of ~0.2 (there are 5 classes)

# Model 2: Static word2vec

Now rather than randomly assign vectors we're going use w2v embeddings. This took me quite a long time to get right, so I'll walk through it line by line

Firstly, we need to load the model. As mentioned I couldn't find the model Kim used, but Kaggle does have a great 200D model trained using GloVe.

In [ ]:
embeddings_index = {}
f = open(os.path.join('../input/glove-global-vectors-for-word-representation', 'glove.6B.200d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

In [ ]:
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Now for the fun bit, we run through all the words in our tokenizer and look for them in the w2v model. If they exist we add them to our embedding matrix (ie our embeddings) if they don't then we assign a random vector

In [ ]:
num_words = min(max_features, len(word_index)) + 1
print(num_words)

# first create a matrix of zeros, this is our embedding matrix
embedding_matrix = np.zeros((num_words, embedding_dim))

# for each word in out tokenizer lets try to find that work in our w2v model
for word, i in word_index.items():
    if i > max_features:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # we found the word - add that words vector to the matrix
        embedding_matrix[i] = embedding_vector
    else:
        # doesn't exist, assign a random vector
        embedding_matrix[i] = np.random.randn(embedding_dim)

Then our model looks almost identical to the first model, except with a different embedding layer

In [ ]:
inputs_2 = Input(shape=(sequence_length,), dtype='int32')

# note the `trainable=False`, later we will make this layer trainable
embedding_layer_2 = Embedding(num_words,
                            embedding_dim,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=sequence_length,
                            trainable=False)(inputs_2)

reshape_2 = Reshape((sequence_length, embedding_dim, 1))(embedding_layer_2)

conv_0_2 = Conv2D(num_filters, kernel_size=(3, embedding_dim), activation='relu', kernel_regularizer=regularizers.l2(3))(reshape_2)
conv_1_2 = Conv2D(num_filters, kernel_size=(4, embedding_dim), activation='relu', kernel_regularizer=regularizers.l2(3))(reshape_2)
conv_2_2 = Conv2D(num_filters, kernel_size=(5, embedding_dim), activation='relu', kernel_regularizer=regularizers.l2(3))(reshape_2)

maxpool_0_2 = MaxPool2D(pool_size=(sequence_length - 3 + 1, 1), strides=(1,1), padding='valid')(conv_0_2)
maxpool_1_2 = MaxPool2D(pool_size=(sequence_length - 4 + 1, 1), strides=(1,1), padding='valid')(conv_1_2)
maxpool_2_2 = MaxPool2D(pool_size=(sequence_length - 5 + 1, 1), strides=(1,1), padding='valid')(conv_2_2)

concatenated_tensor_2 = Concatenate(axis=1)([maxpool_0_2, maxpool_1_2, maxpool_2_2])
flatten_2 = Flatten()(concatenated_tensor_2)

dropout_2 = Dropout(0.5)(flatten_2)
output_2 = Dense(units=5, activation='softmax')(dropout_2)

In [ ]:
model_2 = Model(inputs=inputs_2, outputs=output_2)
model_2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model_2.summary())

In [ ]:
batch_size = 32
history_2 = model_2.fit(X_train, y_train, epochs=30, batch_size=batch_size, verbose=1, validation_split=0.2)

In [ ]:
plt.plot(history_2.history['acc'])
plt.plot(history_2.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

plt.plot(history_2.history['loss'])
plt.plot(history_2.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
y_hat_2 = model_2.predict(X_test)

In [ ]:
accuracy_score(list(map(lambda x: np.argmax(x), y_test)), list(map(lambda x: np.argmax(x), y_hat_2)))

In [ ]:
confusion_matrix(list(map(lambda x: np.argmax(x), y_test)), list(map(lambda x: np.argmax(x), y_hat_2)))

So you can see this model doesn't do that much better (maybe a bit worse depending how this run went) than the random initialization. This agrees with what Kim found.

# Model 3: w2v with trainable embeddings

For this model we're going to try the same model again, but this time make the embeddings trainable. That means if during training the model decides on a better embedding for a word then it'll update it

This has the added benifit of updating the words which were randomly assigned a vector (because they weren't in the w2v model)

In [ ]:
inputs_3 = Input(shape=(sequence_length,), dtype='int32')
embedding_layer_3 = Embedding(num_words,
                            embedding_dim,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=sequence_length,
                            trainable=True)(inputs_3)

reshape_3 = Reshape((sequence_length, embedding_dim, 1))(embedding_layer_3)

# note the relu activation
conv_0_3 = Conv2D(num_filters, kernel_size=(3, embedding_dim), activation='relu', kernel_regularizer=regularizers.l2(3))(reshape_3)
conv_1_3 = Conv2D(num_filters, kernel_size=(4, embedding_dim), activation='relu', kernel_regularizer=regularizers.l2(3))(reshape_3)
conv_2_3 = Conv2D(num_filters, kernel_size=(5, embedding_dim), activation='relu', kernel_regularizer=regularizers.l2(3))(reshape_3)

maxpool_0_3 = MaxPool2D(pool_size=(sequence_length - 3 + 1, 1), strides=(1,1), padding='valid')(conv_0_3)
maxpool_1_3 = MaxPool2D(pool_size=(sequence_length - 4 + 1, 1), strides=(1,1), padding='valid')(conv_1_3)
maxpool_2_3 = MaxPool2D(pool_size=(sequence_length - 5 + 1, 1), strides=(1,1), padding='valid')(conv_2_3)

concatenated_tensor_3 = Concatenate(axis=1)([maxpool_0_3, maxpool_1_3, maxpool_2_3])
flatten_3 = Flatten()(concatenated_tensor_3)

dropout_3 = Dropout(0.5)(flatten_3)
output_3 = Dense(units=5, activation='softmax')(dropout_3)

In [ ]:
model_3 = Model(inputs=inputs_3, outputs=output_3)
model_3.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model_3.summary())

In [ ]:
batch_size = 32
history_3 = model_3.fit(X_train, y_train, epochs=30, batch_size=batch_size, verbose=1, validation_split=0.2)

In [ ]:
plt.plot(history_3.history['acc'])
plt.plot(history_3.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

plt.plot(history_3.history['loss'])
plt.plot(history_3.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
y_hat_3 = model_3.predict(X_test)

In [ ]:
accuracy_score(list(map(lambda x: np.argmax(x), y_test)), list(map(lambda x: np.argmax(x), y_hat_3)))

In [ ]:
confusion_matrix(list(map(lambda x: np.argmax(x), y_test)), list(map(lambda x: np.argmax(x), y_hat_3)))

As you can see this performs much better than that random vector and static w2v models - by 10% which is a massive improvement!

# Model 4: Again, but with binary classification (similar to SST2)

Kim tried his models against a varity of datasets. Two of these datasets are closely related to one another (and similar to our dataset).

SST1 is a semantic dataset release by Stanford. It has 5 categories of sentiment (like our data). SST2 is the same dataset, but with very negative and slightly negative merged into one class and slightly positive and positive also merged (neutral is dropped). This gives us a binary classification we can replicate with our data

In [ ]:
# SST2 is the same as SST1, but with neutrals removed and binary labels
sst2_data = pd.concat([df_0.head(sample_size), df_1.head(sample_size), df_3.head(sample_size), df_4.head(sample_size)]).sample(frac=1)

def merge_sentiments(x):
    if x == 0 or x == 1:
        return 0
    else:
        return 1

sst2_data['Sentiment'] = sst2_data['Sentiment'].apply(merge_sentiments)

sst2_tokenizer = Tokenizer(num_words=max_features, split=' ', oov_token='<unw>')
sst2_tokenizer.fit_on_texts(sst2_data['Phrase'].values)

sst2_X = sst2_tokenizer.texts_to_sequences(sst2_data['Phrase'].values)
sst2_X = pad_sequences(sst2_X, sequence_length)

sst2_y = sst2_data['Sentiment'].values

sst2_X_train, sst2_X_test, sst2_y_train, sst2_y_test = train_test_split(sst2_X, sst2_y, test_size=0.1)

Now lets try to train our model again with this dataset

In [ ]:
inputs_4 = Input(shape=(sequence_length,), dtype='int32')
embedding_layer_4 = Embedding(num_words,
                            embedding_dim,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=sequence_length,
                            trainable=True)(inputs_4)

reshape_4 = Reshape((sequence_length, embedding_dim, 1))(embedding_layer_4)

conv_0_4 = Conv2D(num_filters, kernel_size=(3, embedding_dim), padding='valid', kernel_initializer='normal', activation='relu', kernel_regularizer=regularizers.l2(3))(reshape_4)
conv_1_4 = Conv2D(num_filters, kernel_size=(4, embedding_dim), padding='valid', kernel_initializer='normal', activation='relu', kernel_regularizer=regularizers.l2(3))(reshape_4)
conv_2_4 = Conv2D(num_filters, kernel_size=(5, embedding_dim), padding='valid', kernel_initializer='normal', activation='relu', kernel_regularizer=regularizers.l2(3))(reshape_4)

maxpool_0_4 = MaxPool2D(pool_size=(sequence_length - 3 + 1, 1), strides=(1,1), padding='valid')(conv_0_4)
maxpool_1_4 = MaxPool2D(pool_size=(sequence_length - 4 + 1, 1), strides=(1,1), padding='valid')(conv_1_4)
maxpool_2_4 = MaxPool2D(pool_size=(sequence_length - 5 + 1, 1), strides=(1,1), padding='valid')(conv_2_4)

concatenated_tensor_4 = Concatenate(axis=1)([maxpool_0_4, maxpool_1_4, maxpool_2_4])
flatten_4 = Flatten()(concatenated_tensor_4)

dropout_4 = Dropout(0.5)(flatten_4)
# note the different activation
output_4 = Dense(units=1, activation='sigmoid')(dropout_4)

In [ ]:
model_4 = Model(inputs=inputs_4, outputs=output_4)

# note we're using binary_crossentropy here instead of categorical
model_4.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model_4.summary())

In [ ]:
batch_size = 32
history_4 = model_4.fit(sst2_X_train, sst2_y_train, epochs=30, batch_size=batch_size, verbose=1, validation_split=0.2)

In [ ]:
plt.plot(history_4.history['acc'])
plt.plot(history_4.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

plt.plot(history_4.history['loss'])
plt.plot(history_4.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
y_hat_4 = model_4.predict(sst2_X_test)

accuracy_score(sst2_y_test, list(map(lambda v: v > 0.5, y_hat_4)))

In [ ]:
confusion_matrix(sst2_y_test, list(map(lambda v: v > 0.5, y_hat_4)))

# Conclusions

Here's the final results

In [ ]:
print("CNN random       : " + str(accuracy_score(list(map(lambda x: np.argmax(x), y_test)), list(map(lambda x: np.argmax(x), y_hat)))))
print("CNN static       : " + str(accuracy_score(list(map(lambda x: np.argmax(x), y_test)), list(map(lambda x: np.argmax(x), y_hat_2)))))
print("CNN trainable    : " + str(accuracy_score(list(map(lambda x: np.argmax(x), y_test)), list(map(lambda x: np.argmax(x), y_hat_3)))))
print("Binary trainable : " + str(accuracy_score(sst2_y_test, list(map(lambda v: v > 0.5, y_hat_4)))))

As you can see the trainable embeddings outperform the other models by quite some way. These results are pretty close to what Kim achieved

### Things I learnt

I learnt a lot doing this and it was well worth a weekend's hacking! I will definitely try to do more of this

Firstly, it took me a long time to get anything other than random success with the w2v embeddings. My mistake was to try to clean the data in a way that didn't match the word2vec model at all. This was compounded by allocating a zero vector for words not found, the result was most of my text was labelled with the same vector (0s in 200D) and nothing was learnt

So first and second take away: look at your w2v model and **make sure you don't assign unknown words to the same vector**

Secondly, as I said above I was really surprised how poorly the static w2v embedding worked. I am still not sure exactly why this would be but it might be interesting to investigate.

Finally the big thing learnt here is that you should make your embeddings trainable if you can. It make a huge improvement to this model!


# Thanks and acknowledgements

Obviously, thanks to Yoon Kim for writing such a [detailed and approachable paper](http://www.aclweb.org/anthology/D14-1181). I honestly didn't expect to be able to implement something to easily or achieve such good results.

Joshua Kim wrote an [excellent blog post](http://www.joshuakim.io/understanding-how-convolutional-neural-network-cnn-perform-text-classification-with-word-embeddings/) explaining Yoon Kim's paper, it's well worth a read.

I also found [this projct](https://github.com/bhaveshoswal/CNN-text-classification-keras/blob/master/model.py) on github which helped **a lot** when actually trying to implement this model

As mentioned, I struggled to get the word embeddings working at first but I wouldn't have even go to that point without this [blog post by Keras](https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html) showing you how to use word embeddings. There is also a github repo with more detail

# Lastly

If you've read this far and liked this kernel or have a question/comment/nice idea for making this better - please let me know or use the upvote thing.  It would mean a lot to me!